In [2]:
!wget https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt -O ./Data/irish-lyrics-eof.txt

--2021-05-26 19:29:28--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.195.48, 142.250.195.112, 142.250.196.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.195.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘./Data/irish-lyrics-eof.txt’

./Data/irish-lyrics 100%[===================>]  67.35K  --.-KB/s    in 0.09s   

2021-05-26 19:29:29 (790 KB/s) - ‘./Data/irish-lyrics-eof.txt’ saved [68970/68970]



In [3]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [80]:
poem_path = "./Data/irish-lyrics-eof.txt"
embedding_dim = 100

In [6]:
with open(poem_path, 'r') as poem_file:
    poem_data = poem_file.readlines()

print("No of lines in the poem : ", len(poem_data))

No of lines in the poem :  1692


In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poem_data)
word_index = tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [23]:
training_data = []
max_length = 0

for line in poem_data:
    sequence = tokenizer.texts_to_sequences([line])[0]
    if len(sequence) > max_length:
        max_length = len(sequence)
    for seq_index in range(1, len(sequence)):
        training_data.append(sequence[:seq_index+1])
print("Train data size : ", len(training_data))

Train data size :  12038


In [25]:
print(training_data[:5])
padded_train_data = pad_sequences(training_data, maxlen=max_length, padding="pre")
print(padded_train_data[:5])

[[51, 12], [51, 12, 96], [51, 12, 96, 1223], [51, 12, 96, 1223, 48], [51, 12, 96, 1223, 48, 2]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    51   12]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   51
    12   96]
 [   0    0    0    0    0    0    0    0    0    0    0    0   51   12
    96 1223]
 [   0    0    0    0    0    0    0    0    0    0    0   51   12   96
  1223   48]
 [   0    0    0    0    0    0    0    0    0    0   51   12   96 1223
    48    2]]


In [26]:
train_sentence, train_label = padded_train_data[:, :-1], padded_train_data[:, -1]

print(train_sentence[0])
print(train_label[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 51]
12


In [28]:
train_label_hot_encoded = tf.one_hot(train_label, len(word_index))

In [81]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index)+1, embedding_dim, input_length=max_length-1),
    # tf.keras.layers.Bidirectional(tf.keras.lay ers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(train_label_hot_encoded.shape[1], activation='softmax')
])

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 15, 100)           270000    
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 300)               301200    
_________________________________________________________________
dense_16 (Dense)             (None, 2689)              809389    
Total params: 1,380,589
Trainable params: 1,380,589
Non-trainable params: 0
_________________________________________________________________


In [82]:
history = model.fit(train_sentence, train_label_hot_encoded, epochs=100, verbose=1)

Epoch 1/100
377/377 [==============================] - 18s 40ms/step - loss: 6.9767 - accuracy: 0.0626
Epoch 2/100
377/377 [==============================] - 26s 70ms/step - loss: 6.2507 - accuracy: 0.0702
Epoch 3/100
377/377 [==============================] - 32s 86ms/step - loss: 5.9897 - accuracy: 0.0805
Epoch 4/100
377/377 [==============================] - 43s 115ms/step - loss: 5.6888 - accuracy: 0.0999
Epoch 5/100
377/377 [==============================] - 33s 88ms/step - loss: 5.3265 - accuracy: 0.1117
Epoch 6/100
377/377 [==============================] - 33s 87ms/step - loss: 4.9668 - accuracy: 0.1334
Epoch 7/100
377/377 [==============================] - 32s 86ms/step - loss: 4.6114 - accuracy: 0.1585
Epoch 8/100
377/377 [==============================] - 33s 89ms/step - loss: 4.2229 - accuracy: 0.1872
Epoch 9/100
377/377 [==============================] - 31s 82ms/step - loss: 3.9057 - accuracy: 0.2274
Epoch 10/100
377/377 [==============================] - 32s 85ms/step - 

In [83]:
seed_text = "I've got a bad feeling about this"
next_word = 100

for _ in range(next_word):
    sequence = tokenizer.texts_to_sequences([seed_text])[0]
    padded_seq = pad_sequences([sequence], maxlen=max_length-1, padding='pre')
    predicted_class = model.predict_classes(padded_seq)[0]
    seed_text += " " + reverse_word_index[predicted_class]
print("Output : ")
print(seed_text)

Output : 
I've got a bad feeling about this dead more more shook to town to the coat almost doubts even green and the young ones britches all green were the wind light and play on the rocky farmers daughter and the approaching tree to the town of help the wind maiden more more they might see but more more three dancing low be wear tuam fingers him quite hosannahs to low on beauty you were low more slower of dream inside gold of gold of gold and gold and deep shall only this beauty fly with your say visions for a tired sinking almost nature carrigfergus almost nature
